In [1]:
from baza_mip import mip_session, sessionmaker
from baza_mip.models import Procesy_Przydzielone, Procesy
from sqlalchemy import create_engine, text
from Plan_Pracy.plan_pracy import kj_paczki

import pandas as pd


gen_engine = create_engine("sqlite:///Z:/450. PLANISTA - ZAOPATRZENIE/GENERATORY.db", echo=False)



In [2]:
# from sqlalchemy import URL
# url_obj = URL.create("mysql+mysqlconnector",
#                      username="root",
#                      password="password",
#                      host="192.168.2.213",
#                      port="3306",
#                      database="BAZA_MIP"
#                      )

# ubn_engine = create_engine(url_obj, echo=False)
# Session = sessionmaker(bind=ubn_engine)
# ubn_session = Session()

# ubn_session.query(Procesy.proces).all()

In [3]:
#dostawy
with gen_engine.begin() as conn:
    suma_paczek = conn.execute(text("select SUM(ILE_ZAMOWIONE) from ZAM_PIANKI where nr_SAMOCHODU == 'PIANPOL 15_24'")).fetchall()[0][0]
    zamowione_paczki = conn.execute(text("select NR_KOMPLETACJI, OPIS, ILE_ZAMOWIONE, ZAM1, ZAM2, NR_PARTII from ZAM_PIANKI where nr_SAMOCHODU == 'PIANPOL 15_24'")).fetchall()
    owaty = conn.execute(text("select NR_KOMPLETACJI, OPIS, ILE_ZAMOWIONE, NR_PARTII from ZAM_PIANKI where nr_SAMOCHODU == 'PIANPOL 17_24' AND MODEL == 'WILLOW'")).fetchall()
    owaty_mb = conn.execute(text("select * from OWATY")).fetchall()

In [4]:
mb_owaty = pd.DataFrame(owaty_mb)
mb_owaty

,NAZWA_UKL,OPIS,RODZAJ_CIECIA,ZUZYCIE,TYP_OWATY
0,O3-AMA-25-SP,"AMALFI 2,5",K,3.54,O3
1,O3-AMA-XLA-SP,AMALFI [LA = LA],K,4.38,O3
2,O3-AMA-XRS-SP,AMALFI [RS = RS],K,6.04,O3
3,O3-AMA-X-SP,AMALFI ][,K,0.96,O3
4,O3-AMA-15-SP,"AMALFI 3,5",K,3.54,O3
...,...,...,...,...,...
469,W3,GOYA P112x100,R,2.50,W3
470,W3,GOYA P112x80,R,2.50,W3
471,O1-OVA-X1X-SP,OVAL [1o],K,2.10,O1
472,O1-LEN-X1X-SP,LENOX [1o],K,1.90,O1


In [5]:
def dodaj_KJ(zamowione_paczki):
    czas_calkowity = 0

    for row in zamowione_paczki:
        # print(row[0], row[1], row[2], row[3], row[4], row[5])
        
        print(f"KJ PARTII {row[5]}: {row[1]} - {row[2]}| NR_DOS {row[3]}", ",", kj_paczki(row[2])*10)
        czas_calkowity += kj_paczki(row[2])*10
    
    print("Całkowity czas KJ:",czas_calkowity)

In [6]:
def dodaj_OWATY_ROZKROJ(owaty):
    
    for row in owaty:
        # print(row[0], row[1], row[2], row[3], row[4])
        czas_ciecia = int(mb_owaty[mb_owaty.OPIS == row[1]].ZUZYCIE.sum() * 1.5)
        print(f"WYCINANIE OWAT NR PARTII {row[3]}: {row[1]} - {row[2]}szt| KOMPLETACJA {row[0]},", czas_ciecia)
 

In [7]:
def dodaj_pakowanie(zamowione_paczki):
    czas_calkowity = 0
    pakowanie = list()
    for row in zamowione_paczki:
        # print(row[0], row[1], row[2], row[3], row[4], row[5])
        
        print(f"PAKOWANIE PARTII {row[5]}: {row[1]} - {row[2]}| KOMPLETACJA {row[0]}", ",", row[2])
        czas_calkowity += row[2]
        pakowanie.append([f"PAKOWANIE PARTII {row[5]}: {row[1]} - {row[2]}| KOMPLETACJA {row[0]}", ",", row[2]])
    
    print("Całkowity czas KJ:",czas_calkowity)

In [8]:
suma_paczek

505

In [9]:
zamowione_paczki

[('4_24', 'CALYPSO 4', 20, '24/0637', '', '25/01'),
 ('2_24', 'ELIXIR 3', 30, '24/0637', '', '25/01'),
 ('2_24', 'ELIXIR 4', 40, '24/0637', '', '25/01'),
 ('2_24', 'ELIXIR N', 60, '24/0637', '', '25/01'),
 ('2_24', 'ELIXIR ][', 150, '24/0637', '', '25/01'),
 ('4_24', 'REVERSO 1,5', 40, '24/0637', '', '25/01'),
 ('4_24', 'REVERSO 2', 25, '24/0637', '', '25/01'),
 ('4_24', 'REVERSO N106x106', 40, '24/0637', '', '25/01'),
 ('4_24', 'REVERSO ][', 40, '24/0637', '', '25/01'),
 ('4_24', 'REVERSO ][ - L', 40, '24/0637', '', '25/01'),
 ('3_24', 'MAXWELL Ons', 20, '24/0637', '', '25/01')]

In [10]:
dodaj_pakowanie(zamowione_paczki)

PAKOWANIE PARTII 25/01: CALYPSO 4 - 20| KOMPLETACJA 4_24 , 20
PAKOWANIE PARTII 25/01: ELIXIR 3 - 30| KOMPLETACJA 2_24 , 30
PAKOWANIE PARTII 25/01: ELIXIR 4 - 40| KOMPLETACJA 2_24 , 40
PAKOWANIE PARTII 25/01: ELIXIR N - 60| KOMPLETACJA 2_24 , 60
PAKOWANIE PARTII 25/01: ELIXIR ][ - 150| KOMPLETACJA 2_24 , 150
PAKOWANIE PARTII 25/01: REVERSO 1,5 - 40| KOMPLETACJA 4_24 , 40
PAKOWANIE PARTII 25/01: REVERSO 2 - 25| KOMPLETACJA 4_24 , 25
PAKOWANIE PARTII 25/01: REVERSO N106x106 - 40| KOMPLETACJA 4_24 , 40
PAKOWANIE PARTII 25/01: REVERSO ][ - 40| KOMPLETACJA 4_24 , 40
PAKOWANIE PARTII 25/01: REVERSO ][ - L - 40| KOMPLETACJA 4_24 , 40
PAKOWANIE PARTII 25/01: MAXWELL Ons - 20| KOMPLETACJA 3_24 , 20
Całkowity czas KJ: 505


In [11]:
dodaj_KJ(zamowione_paczki)

KJ PARTII 25/01: CALYPSO 4 - 20| NR_DOS 24/0637 , 20
KJ PARTII 25/01: ELIXIR 3 - 30| NR_DOS 24/0637 , 30
KJ PARTII 25/01: ELIXIR 4 - 40| NR_DOS 24/0637 , 40
KJ PARTII 25/01: ELIXIR N - 60| NR_DOS 24/0637 , 50
KJ PARTII 25/01: ELIXIR ][ - 150| NR_DOS 24/0637 , 100
KJ PARTII 25/01: REVERSO 1,5 - 40| NR_DOS 24/0637 , 40
KJ PARTII 25/01: REVERSO 2 - 25| NR_DOS 24/0637 , 30
KJ PARTII 25/01: REVERSO N106x106 - 40| NR_DOS 24/0637 , 40
KJ PARTII 25/01: REVERSO ][ - 40| NR_DOS 24/0637 , 40
KJ PARTII 25/01: REVERSO ][ - L - 40| NR_DOS 24/0637 , 40
KJ PARTII 25/01: MAXWELL Ons - 20| NR_DOS 24/0637 , 20
Całkowity czas KJ: 450


In [12]:
kjoty = [["KJ PARTII 25/01: CALYPSO 4 - 20| NR_DOS 24/0637" , 20],
            ["KJ PARTII 25/01: ELIXIR 3 - 30| NR_DOS 24/0637" , 30],
            ["KJ PARTII 25/01: ELIXIR 4 - 40| NR_DOS 24/0637" , 40],
            ["KJ PARTII 25/01: ELIXIR N - 60| NR_DOS 24/0637" , 50],
            ["KJ PARTII 25/01: ELIXIR ][ - 150| NR_DOS 24/0637" , 100],
            ["KJ PARTII 25/01: REVERSO 1,5 - 40| NR_DOS 24/0637" , 40],
            ["KJ PARTII 25/01: REVERSO 2 - 25| NR_DOS 24/0637" , 30],
            ["KJ PARTII 25/01: REVERSO N106x106 - 40| NR_DOS 24/0637" , 40],
            ["KJ PARTII 25/01: REVERSO ][ - 40| NR_DOS 24/0637" , 40],
            ["KJ PARTII 25/01: REVERSO ][ - L - 40| NR_DOS 24/0637" , 40],
            ["KJ PARTII 25/01: MAXWELL Ons - 20| NR_DOS 24/0637" , 20]]

pakowanie =[
    ["PAKOWANIE PARTII 25/01: CALYPSO 4 - 20| KOMPLETACJA 4_24" , 20],
    ["PAKOWANIE PARTII 25/01: ELIXIR 3 - 30| KOMPLETACJA 2_24" , 30],
    ["PAKOWANIE PARTII 25/01: ELIXIR 4 - 40| KOMPLETACJA 2_24" , 40],
    ["PAKOWANIE PARTII 25/01: ELIXIR N - 60| KOMPLETACJA 2_24" , 60],
    ["PAKOWANIE PARTII 25/01: ELIXIR ][ - 150| KOMPLETACJA 2_24" , 150],
    ["PAKOWANIE PARTII 25/01: REVERSO 1,5 - 40| KOMPLETACJA 4_24" , 40],
    ["PAKOWANIE PARTII 25/01: REVERSO 2 - 25| KOMPLETACJA 4_24" , 25],
    ["PAKOWANIE PARTII 25/01: REVERSO N106x106 - 40| KOMPLETACJA 4_24" , 40],
    ["PAKOWANIE PARTII 25/01: REVERSO ][ - 40| KOMPLETACJA 4_24" , 40],
    ["PAKOWANIE PARTII 25/01: REVERSO ][ - L - 40| KOMPLETACJA 4_24" , 40],
    ["PAKOWANIE PARTII 25/01: MAXWELL Ons - 20| KOMPLETACJA 3_24" , 20],
]

kjoty = [["KJ PARTII 25/01: CALYPSO 4 - 20| NR_DOS 24/0637" , 20],
            ["KJ PARTII 25/01: ELIXIR 3 - 30| NR_DOS 24/0637" , 30],
            ["KJ PARTII 25/01: ELIXIR 4 - 40| NR_DOS 24/0637" , 40],
            ["KJ PARTII 25/01: ELIXIR N - 60| NR_DOS 24/0637" , 50],
            ["KJ PARTII 25/01: ELIXIR ][ - 150| NR_DOS 24/0637" , 100],
            ["KJ PARTII 25/01: REVERSO 1,5 - 40| NR_DOS 24/0637" , 40],
            ["KJ PARTII 25/01: REVERSO 2 - 25| NR_DOS 24/0637" , 30],
            ["KJ PARTII 25/01: REVERSO N106x106 - 40| NR_DOS 24/0637" , 40],
            ["KJ PARTII 25/01: REVERSO ][ - 40| NR_DOS 24/0637" , 40],
            ["KJ PARTII 25/01: REVERSO ][ - L - 40| NR_DOS 24/0637" , 40],
            ["KJ PARTII 25/01: MAXWELL Ons - 20| NR_DOS 24/0637" , 20]]

pakowanie =[
    ["PAKOWANIE PARTII 25/01: CALYPSO 4 - 20| KOMPLETACJA 4_24" , 20],
    ["PAKOWANIE PARTII 25/01: ELIXIR 3 - 30| KOMPLETACJA 2_24" , 30],
    ["PAKOWANIE PARTII 25/01: ELIXIR 4 - 40| KOMPLETACJA 2_24" , 40],
    ["PAKOWANIE PARTII 25/01: ELIXIR N - 60| KOMPLETACJA 2_24" , 60],
    ["PAKOWANIE PARTII 25/01: ELIXIR ][ - 150| KOMPLETACJA 2_24" , 150],
    ["PAKOWANIE PARTII 25/01: REVERSO 1,5 - 40| KOMPLETACJA 4_24" , 40],
    ["PAKOWANIE PARTII 25/01: REVERSO 2 - 25| KOMPLETACJA 4_24" , 25],
    ["PAKOWANIE PARTII 25/01: REVERSO N106x106 - 40| KOMPLETACJA 4_24" , 40],
    ["PAKOWANIE PARTII 25/01: REVERSO ][ - 40| KOMPLETACJA 4_24" , 40],
    ["PAKOWANIE PARTII 25/01: REVERSO ][ - L - 40| KOMPLETACJA 4_24" , 40],
    ["PAKOWANIE PARTII 25/01: MAXWELL Ons - 20| KOMPLETACJA 3_24" , 20],
]

owaty = [    
    ["WYCINANIE OWAT NR PARTII 26/02: WILLOW 1 - 40szt| KOMPLETACJA 3_24", 2],
    ["WYCINANIE OWAT NR PARTII 26/02: WILLOW 1,5 - 40szt| KOMPLETACJA 3_24", 3],
    ["WYCINANIE OWAT NR PARTII 26/02: WILLOW [L2Hz - 20szt| KOMPLETACJA 3_24", 14],
    ["WYCINANIE OWAT NR PARTII 26/02: WILLOW [ - 80szt| KOMPLETACJA 3_24", 3],
    ["WYCINANIE OWAT NR PARTII 26/02: WILLOW ] - 60szt| KOMPLETACJA 3_24", 0]

]


In [13]:
##eliza 5
##adamr 4
##piotrl 8


# mip_session.add(Procesy_Przydzielone(4, 5, 1, "DOSTAWA PIANPOL 15_24 PARTIA: 25/01", "2024-08-02", 505))

# for kj in kjoty:
#     mip_session.add(Procesy_Przydzielone(4, 5, 2, kj[0], "2024-08-02", kj[1]))

# for p in pakowanie:
#     mip_session.add(Procesy_Przydzielone(4, 5, 3, p[0], "2024-08-02", p[1]))

# mip_session.add(Procesy_Przydzielone(4, 5, 11, "DOSTAWA OWAT Z:50, N:20, C:80", "2024-08-05", 150*9))

for p in owaty:

    mip_session.add(Procesy_Przydzielone(4, 5, 9, p[0], "2024-08-02", p[1]))


mip_session.commit()

In [14]:
mip_session.execute(text("SELECT * FROM baza_mip.procesy_przydzielone")).all()

[(1, 4, 5, 2, 'KONTROLA JAKOSCI: HORIZON 1z - 150| NR_DOS 24/0559', '2024-07-09', '2024-07-09', 1, 3, 1, None, 100),
 (2, 4, 5, 2, 'KONTROLA JAKOSCI: HORIZON 1,5z - 150| NR_DOS 24/0559', '2024-07-09', '2024-07-09', 1, 3, 1, None, 100),
 (3, 4, 5, 2, 'KONTROLA JAKOSCI: HORIZON 2,5z - 25| NR_DOS 24/0559', '2024-07-09', '2024-07-09', 1, 0, 1, None, 30),
 (4, 4, 5, 2, 'KONTROLA JAKOSCI: HORIZON LHz][ - 30| NR_DOS 24/0559', '2024-07-09', '2024-07-09', 1, 0, 1, None, 30),
 (5, 4, 5, 2, 'KONTROLA JAKOSCI: HORIZON L2Hz][ - 50| NR_DOS 24/0559', '2024-07-09', '2024-07-09', 1, 0, 1, None, 50),
 (6, 4, 5, 2, 'KONTROLA JAKOSCI: HORIZON P82x82 - 20| NR_DOS 24/0559', '2024-07-09', '2024-07-09', 1, 0, 1, None, 20),
 (7, 4, 5, 2, 'KONTROLA JAKOSCI: HORIZON ][ - 130| NR_DOS 24/0559', '2024-07-09', '2024-07-09', 1, 0, 1, None, 50),
 (8, 4, 5, 2, 'KONTROLA JAKOSCI: GOYA 1 - 20| NR_DOS 24/0583', '2024-07-09', '2024-07-09', 1, 3, 1, None, 20),
 (9, 4, 5, 2, 'KONTROLA JAKOSCI: GOYA 1,5 - 20| NR_DOS 24/0583',